In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"

  # IPython 7.14 deprecated set_matplotlib_formats from IPython
  try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
  except ImportError:
    # Fall back to deprecated location for older IPython versions
    from IPython.display import set_matplotlib_formats
    
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L1VzZXJzL0pvb25LaW1fMS9tbC9wb3N0cy9kYWlseS8yMDI1LTEyLTEw'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/importlib/_bootstrap.py": 1749060473.2193995, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/importlib/_bootstrap_external.py": 1749060473.2249148, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/codecs.py": 1749060466.796989, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/encodings/aliases.py": 1749060474.6800418, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/encodings/__init__.py": 1749060474.6757293, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/encodings/utf_8.py": 1749060475.457727, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/encodings/latin_1.py": 1749060475.2213795, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/abc.py": 1749060466.7068338, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/io.py": 1749060466.9236631, "/usr/local/Caskroom/miniforge/base/envs/pyml/lib/python3.9/stat.py": 1749060467.1213095, "/usr/loca

In [2]:
#| echo: false
#| include: false
import sys
from pathlib import Path


def _ensure_code_on_path(marker="_quarto.yml", code_dir="code"):
    """Add the shared code directory to sys.path when rendering notes."""
    cwd = Path.cwd().resolve()
    for candidate in (cwd, *cwd.parents):
        if (candidate / marker).exists():
            code_path = candidate / code_dir
            if code_path.exists() and str(code_path) not in sys.path:
                sys.path.insert(0, str(code_path))
            return
    raise RuntimeError(f"Could not locate {marker} above {cwd}")


_ensure_code_on_path()

In [3]:
class AdalineGD:
    """ADAptive LInear NEuron classifier.

    Parameters
    --------------

    eta: float. 
        This is the learning rate (between 0.0 and 1.0)
    n_iter: int
        Passes over the training dataset
    random_state: int
        Random number generator seed for random weight generalization
    
    Attributes
    ---------------
    w_: 1D-Array
        Weights after fitting
    b_: Scalar
        Bias after fitting
    losses_: list
        Mean Squared Error loss function values in each epoch
    """
    def __init__(self, eta=0.01, n_iter = 50, random_state = 1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def net_input(self, X):
        """Calculate net input
        
        Think of this as the linear combination of features and weights + bias
        This helps get the neuron's "raw score"
        """
        return np.dot(X, self.w_) + self.b_

    def activation(self, X):
        """Compute linear activation
        
        This is just the identity."""
        return X
    
    def predict(self, X): 
        """Return class label after unit step
        
        Binary class label of 1 when the activation >= 0.5, else 0
        """
        return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        -----------
        X: array-like, shape = [m_examples, n_features]
            Training vectors
            m_examples: # of examples
            n_features: # of features
        
        y: array-like, shape = [m_examples]
            Target values
        
        Returns
        -----------
        self: object
        """

        # Initialize the RNG, weights w_ with small random values, bias b_ to zero, losses_ empty
        rgen = np.random.RandomState(self.random_state)
        # This starts weights as tiny random numbers
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1]) 
        # This starts the bias at zero
        self.b_ = np.float_(0.)
        # This prepares a list to record training error at each pass
        self.losses_ = []

        for i in range(self.n_iter): # for each epoch
            # Given whatever is w_ and b_, we're computing the raw score for each example
            net_input = self.net_input(X) 
            # Activation is the identity (no "squashing" yet)
            output = self.activation(net_input)
            # How far off are we from the desired targets? 
            # Positive if too low, negative if too high
            errors = (y - output)
            
            """Weight update. 
            
            This step is important!
            We are calculating the gradient based on the whole training set,
            not just evaluating each individual training example (as in the perceptron).
            This makes the learning "smoother" - less aberrations because of individual examples.

            This is called "Batch Gradient Descent." 
            
            Think of the model as making guesses with two kinds of knobs:
            w_: one knob per input feature (like volume sliders for each input).
            b_: one extra knob that shifts everything up or down (like a master volume).

            Weight update (the "averaged nudge"):

            Recall that error = (y - (X * w + b))
            Multiplying by * 2.0 is because of the Chain Rule
            The loss is error^2, so the derivative is 2 * error * derivative of error
            Derivative of (X * w + b) w.r.t. w is X.
            Multiply X.T * error to aggregate per feature across all samples

            X has rows of training examples, columns of features
            errors is rows of how wrong we were per example
            X.T is the transpose of X so that each feature lines up with the 
                errors across example
            X.T.dot(errors) is the dot product that combines every feature with its errors. 
            X is (n_samples, n_features); errors is (n_samples,). 
            Flipping X gives X.T as (n_features, n_samples).
            Dotting (n_features, n_samples) with (n_samples,) yields (n_features,):
                a separate summed value for each feature.
            
            Recall that the loss is the mean squared error
            Therefore, we need to divide by N (# of examples) so the summed gradient becomes an average
            / X.shape[0] means “take the average over all examples” so we dont overreact to any single case.

            Bias update: nudge the bias by the avg error
            """
            self.w_ += self.eta * 2.0 * X.T.dot(errors) / X.shape[0]
            self.b_ += self.eta * 2.0 * errors.mean()
            # Compute Mean Squared Error
            loss = (errors**2).mean()
            # Track loss history so we can see learning progress
            self.losses_.append(loss)
        return self